Imports

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


Load and Sample Data

In [5]:
# Load CSV
df = pd.read_csv('/Users/caitlyndinh/ai-academy/FinalProject/merged_with_naics.csv', low_memory=False)

# Sample to reduce memory use
df = df.sample(n=50000, random_state=42)  # Change sample size if needed
df.shape


(50000, 43)

Create Target Variable

In [6]:
df['Default'] = df['ChargeOffDate'].notna().astype(int)
df = df.dropna(subset=['Default'])
df['Default'].value_counts()


Default
0    44229
1     5771
Name: count, dtype: int64

Drop Leakage Columns

In [7]:
leakage_cols = ['ChargeOffDate', 'GrossChargeOffAmount', 'PaidInFullDate', 'LoanStatus', 'FirstDisbursementDate']
df = df.drop(columns=[col for col in leakage_cols if col in df.columns])


Feature Selection

In [8]:
features = [
    'Program', 'Subprogram', 'GrossApproval', 'SBAGuaranteedApproval', 'ApprovalDate',
    'InitialInterestRate', 'FixedOrVariableInterestInd', 'TermInMonths',
    'BusinessType', 'BusinessAge', 'CollateralInd', 'RevolverStatus', 'JobsSupported', 'ProjectState'
]

X = df[features].copy()
y = df['Default']


Feature Engineering on Dates

In [9]:
X['ApprovalDate'] = pd.to_datetime(X['ApprovalDate'], errors='coerce')
X['ApprovalYear'] = X['ApprovalDate'].dt.year
X['ApprovalMonth'] = X['ApprovalDate'].dt.month
X = X.drop(columns=['ApprovalDate'])


Define Column Types

In [10]:
numeric_features = ['GrossApproval', 'SBAGuaranteedApproval', 'InitialInterestRate',
                    'TermInMonths', 'JobsSupported', 'ApprovalYear', 'ApprovalMonth']

categorical_features = [col for col in X.columns if col not in numeric_features]


Preprocessing Pipelines

In [19]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()) 
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))  # sparse to save memory
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


Split the Data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


((40000, 15), (10000, 15))

Define Training & Evaluation Function

In [18]:
def train_and_evaluate(model, model_name):
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]

    print(f" {model_name} Results")
    print("=" * 35)
    print(classification_report(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_proba))

train_and_evaluate(LogisticRegression(max_iter=1000), "Logistic Regression")
train_and_evaluate(RandomForestClassifier(n_estimators=100, random_state=42), "Random Forest")
train_and_evaluate(XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42), "XGBoost")


/opt/anaconda3/envs/aiml-academy/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 Logistic Regression Results
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      8846
           1       0.65      0.18      0.28      1154

    accuracy                           0.89     10000
   macro avg       0.78      0.58      0.61     10000
weighted avg       0.87      0.89      0.87     10000

AUC: 0.8443066435063915
 Random Forest Results
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      8846
           1       0.77      0.51      0.61      1154

    accuracy                           0.93     10000
   macro avg       0.85      0.75      0.79     10000
weighted avg       0.92      0.93      0.92     10000

AUC: 0.9409743596475177


/opt/anaconda3/envs/aiml-academy/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:40:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 XGBoost Results
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      8846
           1       0.76      0.70      0.73      1154

    accuracy                           0.94     10000
   macro avg       0.86      0.83      0.85     10000
weighted avg       0.94      0.94      0.94     10000

AUC: 0.9674460467596708
